In [6]:
import os
import sys
import importlib

import pandas as pd
import numpy as np

from scipy.io import mmread

sys.path.append(os.path.abspath("./src"))

import utility as ut
import hessenberg as hg
from variables import *
from qr import QR


pd.set_option("display.max_columns", 10)
pd.set_option("display.width", 1000)
pd.set_option("display.precision", 12)
print("Setup compelete.")


Setup compelete.


# Hessenberg Transform

The Hessenberg transform is a similarity transformation used to reduce a complex matrix to hessenberg form. The following implementation performs the transform using *householder vectors*. The following cells check that for a given matrix $M \in \mathbb{C}^{n \times n}$,
$$M = UHU^{*}$$
where, H is the hessenberg form of $M$ and $U$ is a unitary matrix.

The unittests present in `/tests` check for the equivalence of the eigenvalues of $M$ and $H$. 

## Complex Matrices

In [7]:
a = -20
b = 50
n = 5
m = ut.complex_matrix(n, a, b)
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.conj().T - m, np.zeros((n, n)))}")

Original matrix:
                                   0                                 1                                 2                                 3                                 4
0  16.073493459788+10.073603437667j  19.907746733518+19.146812666067j -17.036828028689+37.346563965095j  45.901097494281-11.180907848347j  16.918617873100+20.898184006594j
1  27.540037900158+33.197156488686j   7.796639387605+41.588886854514j   5.726869408585+46.865996161131j  29.367193371282+14.293216757438j  -8.520867915037+46.091179953550j
2  25.204933742883+49.432686967186j -0.2316704528710-5.4661755637690j -3.1841929030700+8.4583406704900j -11.937104283268+14.365120534397j -0.4214361323950-7.3595064999230j
3 -19.075334242703+28.750312327493j   17.059182125825-8.527876547716j  29.962201966264+23.974239600693j   13.013030072258+0.671861814160j  23.543667752879+18.312533803515j
4   16.583915330048+9.219860483798j  26.396163696409-19.699403312789j  10.181639302081+14.834597918322j  44.967448570971-1

## Real Matrices

### Random Matrices

In [8]:
a = -20
b = 50
n = 5
m = (b - a) * np.random.default_rng().random((n, n)) + a
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((n, n)))}")

Original matrix:
                  0                1                2                3                4
0  -8.918017136949  34.088862589201  27.381416731384  19.500924744176  16.511029316141
1   5.423621711342  -2.861614132498  -4.296769915844  30.529130366580   9.338240785776
2  23.111753023803  -1.406750572064  -0.393093575834  15.905483099960  42.929087373510
3  39.032995547637  23.979124487523  44.437532577470  19.048302958659  15.763220096837
4  38.369669457198   2.533724856220   9.250160343818 -16.756345324223  44.861783686922
Hessenberg transformed:
                  0                1                2                3                4
0  -8.918017136949 -37.083496441256 -29.949458821663  10.411954143945 -13.569381293370
1 -59.660498455657  58.237954454395  -0.615749598972 -35.970393020693   9.850425407622
2   0.000000000000  29.077347486382   0.382358914422  27.710006567421  -0.495811765708
3   0.000000000000   0.000000000000  44.614899182452   3.811774052056 -14.220294576744


### Matrix Market

In [9]:
files = ["west0381", "blckhole"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	h, u = hg.hessenberg_transform(m) 
	print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((m.shape[0], m.shape[0])))}")

Is the transformation similar: True
Is the transformation similar: True


# QR 

For a given matrix $M \in \mathbb{C}^{n\times n}$, in general, the $QR$ algorithm seeks to perform the following iteration:
* $Q_kR_k := M_k$
* $M_{k + 1} := R_kQ_k$

This algorithm can be made more stable and efficient in two ways. The first is to use $M$ is hessenberg form and the second is use to use shifts. When $H$ (hessenberg form of $M$), is used, the $QR$ decompisition of $H$ can be procedurally generated using Givens rotation matrices, $G$ (see the documentation for explanation and generation). The generation of the QR decomposition and then the subsequent formation of $RQ$ takes place as follows
* $R := G_1 G_2 \dots G_k H.$
* $Q := G_1 G_2 \dots G_k.$
* $H_{\text{new}} := R G_{k}^{*} G_{k - 1}^{*} \dots G_{1}^{*} = RQ.$

where $k \leq n - 2$.


## Wilkinson Shift

The Wilkinson shift employs stable shifts to accelerate convergence of the $QR$ hessenberg algorithm. In general the shifted algorithm looks as follows 
* $Q_kR_k := M_k - \sigma I$
* $M_{k + 1} := R_kQ_k + \sigma I$

The shift $\sigma$ is calculated as detailed in the documentation. The $QR$ decomposition and subsequent formation of $RQ$ is done using the hessenberg form of $M$ Givens matrices as shown above.

### Complex Matrices

In [10]:
a = -20
b = 50
n = 5
tol = 1e-8
m = ut.complex_matrix(n, a, b)
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                Eigenvalues (Numpy)
0  96.392487866127+82.888324304038j
1  58.767800940228-22.798970333181j
2  36.008519199485+18.152055587254j
3 -27.289164697473-38.269900533519j
4 -43.048861468545+15.763956958370j
               Eigenvalues (Script)
0  96.392487866127+82.888324304038j
1  58.767800940228-22.798970333181j
2  36.008519199485+18.152055587254j
3 -27.289164697473-38.269900533519j
4 -43.048861468545+15.763956958370j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


### Real Matrices

#### Random Matrices

In [11]:
a = -20
b = 50
n = 5
tol = 1e-8
m = (b - a) * np.random.default_rng().random((n, n)) + a
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                Eigenvalues (Numpy)
0   76.522886347993+0.000000000000j
1   53.560273379421+0.000000000000j
2  0.9366391276240+0.0000000000000j
3 -18.233164827394+16.004279105207j
4 -18.233164827394-16.004279105207j
               Eigenvalues (Script)
0   76.522886347993+0.000000000000j
1   53.560273379421+0.000000000000j
2  0.9366391276240+0.0000000000000j
3 -18.233164827394+16.004279105207j
4 -18.233164827394-16.004279105207j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


In [12]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)
tol = 1e-8
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

   Eigenvalues (Numpy)
0             5.0+6.0j
1             5.0-6.0j
2             4.0+0.0j
3             3.0+0.0j
4             1.0+2.0j
5             1.0-2.0j
   Eigenvalues (Script)
0              5.0+6.0j
1              5.0-6.0j
2              4.0+0.0j
3              3.0+0.0j
4              1.0+2.0j
5              1.0-2.0j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


#### Matrix Market

In [13]:
files = ["gre__115"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	tol = 1e-8
	qr_alg = QR(m)
	u, r = qr_alg.qr_wilkinson_shift(1e-128, 200)
	eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
	eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
	b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
	print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
	if not b:
		with open("output_mm.txt", "w") as f:
			f.write(f"{mm.to_string()}")

Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


## Double Shift

#### Random Matrices

In [14]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)

tol = 1e-8
qr_alg = QR(m)
u, r = qr_alg.double_shift(1e-128, 200)
eigs = np.sort(np.linalg.eig(m)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

   Eigenvalues (Numpy)
0             5.0+6.0j
1             5.0-6.0j
2             4.0+0.0j
3             3.0+0.0j
4             1.0+2.0j
5             1.0-2.0j
   Eigenvalues (Script)
0              5.0+6.0j
1              5.0-6.0j
2              4.0+0.0j
3              3.0+0.0j
4              1.0+2.0j
5              1.0-2.0j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


In [4]:
a = -20
b = 50
n = 5
tol = 1e-8
m = (b - a) * np.random.default_rng().random((n, n)) + a
qr_alg = QR(m)
u, r = qr_alg.double_shift(1e-128, 200)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                Eigenvalues (Numpy)
0   51.419436746842-0.000000000000j
1  22.167285953793+32.133366474526j
2  22.167285953793-32.133366474526j
3  7.7293981794390+8.8070459431140j
4  7.7293981794390-8.8070459431140j
               Eigenvalues (Script)
0   51.419436746843+0.000000000000j
1  22.167285953793+32.133366474526j
2  22.167285953793-32.133366474526j
3  7.7293981794390+8.8070459431140j
4  7.7293981794390-8.8070459431140j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True
